## 36 Terreno com esferas e iluminação com múltiplas fontes de luz 


In [1]:
import glm
import numpy as np
import math
import time
import OpenGL.GL as gl
from PyQt5 import QtOpenGL, QtCore
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.MultiLightPhongShadingShaderProgram import MultiLightPhongShadingShaderProgram
from cg.shader_programs.SimpleShaderProgram_v2 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v3 import ModelRenderer
from cg.models.TerrainMesh_v1 import TerrainMesh
from cg.models.SphereMesh_v2 import SphereMesh

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        self.cameraPos   = glm.vec3(0.0, 2.5, 1.5)
        self.cameraFront = glm.vec3(0.0, 0.0, 0.0)
        self.cameraUp    = glm.vec3(0.0, 1.0,  0.0)
    
        self.viewMatrix = glm.lookAt(self.cameraPos, self.cameraFront, self.cameraUp)
        
        # cria uma malha esférica com raio 0.07 como uma grid de triânguldo de dimensão (20, 20)
        sphere_mesh = SphereMesh(0.07, 20, 20)
        
        # cria uma malha de terreno de tamanho (2.0, 2.0, 0.5) como uma grid de triânguldo de dimensão (300, 300)
        # a malha é criada centrada na orige e paralela ao plano xy (2.0, 2.0) com profundidade/altura 0.5
        # a posição de uma determinada coordenada da grid do terreno 
        # pode ser recuperada com o método getPositionAt(row, col) 
        self.terrainMesh = TerrainMesh('cg/images/heightmap_1.png', 2.0, 2.0, 0.5, 300, 300)
        
        # cria o objeto responsável por carregar os dados para a GPU e renderizá-los
        self.terrainRenderer = ModelRenderer(self.terrainMesh.getVertexPositions(),
                                             vertex_indices=self.terrainMesh.getVertexIndices(),
                                             vertex_normal=self.terrainMesh.getVertexNormals())
        
        self.multiPhongSphereRenderer = ModelRenderer(sphere_mesh.getVertexPositions(),
                                             vertex_indices=sphere_mesh.getVertexIndices(),
                                             vertex_normal=sphere_mesh.getVertexNormals())
        
        self.simpleSphereRenderer = ModelRenderer(sphere_mesh.getVertexPositions(),
                                             vertex_indices=sphere_mesh.getVertexIndices(),
                                             vertex_normal=sphere_mesh.getVertexNormals())
        
        # cria um shader de iluminação Phong e um shader simples
        self.multiPhongShaderProgram = MultiLightPhongShadingShaderProgram()
        self.simpleShaderProgram = SimpleShaderProgram()

        # recupera o endereço da variável de entrada do shader program
        # configura os dados do modelo para serem os dados de entrada do shader program
        position_loc = self.multiPhongShaderProgram.getVertexPositionLoc()
        normal_loc = self.multiPhongShaderProgram.getVertexNormalLoc()
        self.terrainRenderer.setVertexPositionLoc(position_loc)
        self.terrainRenderer.setVertexNormalLoc(normal_loc)
        self.multiPhongSphereRenderer.setVertexPositionLoc(position_loc)
        self.multiPhongSphereRenderer.setVertexNormalLoc(normal_loc)
        
        position_loc = self.simpleShaderProgram.getVertexPositionLoc()
        self.simpleSphereRenderer.setVertexPositionLoc(position_loc)

        # habilita teste de profundidade e culling
        gl.glEnable(gl.GL_DEPTH_TEST);
        gl.glEnable(gl.GL_CULL_FACE)
        
        #inicializa variáveis para animar as esferas
        #basicamente, armazena as coordenada (row, col) da grid do terreno de cada esfera
        self.terrain_index = np.array([[150, 150], [150, 150], [150, 150]], dtype=np.int32)
        self.aux_index = np.array([[150, 150], [150, 150], [150, 150]], dtype=np.int32)
        self.index_offset = np.array([[0, 0], [0, 0], [0, 0]], dtype=np.int32)
        
        # armazena o momento que o programa começou
        self.startTime = time.time()
        
        self.globalTransformation = glm.rotate(glm.mat4(), glm.radians(-90), glm.vec3(1.0, 0.0, 0.0))
        
        # ativa o shader programa para configurar as propriedades da luz
        self.multiPhongShaderProgram.bind()
        
        # permite usar uma cor única para todos os vértices
        # tem o mesmo funcionamento da função useUniformColor do SimpleShader
        self.multiPhongShaderProgram.useUniformMaterialColor(True)
        
        #habilita mais duas fonte de luz
        self.multiPhongShaderProgram.enableLight(1) # index da fonte de luz
        self.multiPhongShaderProgram.enableLight(2) # index da fonte de luz
        
        #configura as cores das três fontes de luz
        self.multiPhongShaderProgram.setUniformLightAmbient(0, np.array([0.0, 0.0, 0.2], dtype=np.float32))
        self.multiPhongShaderProgram.setUniformLightIntensity(0, np.array([0.0, 0.0, 1.0], dtype=np.float32))
        
        self.multiPhongShaderProgram.setUniformLightAmbient(1, np.array([0.0, 0.2, 0.0], dtype=np.float32))
        self.multiPhongShaderProgram.setUniformLightIntensity(1, np.array([0.0, 1.0, 0.0], dtype=np.float32))
        
        self.multiPhongShaderProgram.setUniformLightAmbient(2, np.array([0.2, 0.0, 0.0], dtype=np.float32))
        self.multiPhongShaderProgram.setUniformLightIntensity(2, np.array([1.0, 0.0, 0.0], dtype=np.float32))
        
        #configura a atenuação da luz conforme a distância
        self.multiPhongShaderProgram.useUniformLightAttenuation(0, True)
        self.multiPhongShaderProgram.setUniformLightConstantAttenuation(0, 0.3)
        self.multiPhongShaderProgram.setUniformLightLinearAttenuation(0, 0.1)
        self.multiPhongShaderProgram.setUniformLightQuadraticAttenuation(0, 0.7)
        
        self.multiPhongShaderProgram.useUniformLightAttenuation(1, True)
        self.multiPhongShaderProgram.setUniformLightConstantAttenuation(1, 0.3)
        self.multiPhongShaderProgram.setUniformLightLinearAttenuation(1, 0.1)
        self.multiPhongShaderProgram.setUniformLightQuadraticAttenuation(1, 0.7)
        
        self.multiPhongShaderProgram.useUniformLightAttenuation(2, True)
        self.multiPhongShaderProgram.setUniformLightConstantAttenuation(2, 0.3)
        self.multiPhongShaderProgram.setUniformLightLinearAttenuation(2, 0.1)
        self.multiPhongShaderProgram.setUniformLightQuadraticAttenuation(2, 0.7)
        
        self.multiPhongShaderProgram.release()
        
        self.simpleShaderProgram.bind()
        self.simpleShaderProgram.useUniformColor(True)
        self.simpleShaderProgram.release()
        
        # habilita a mistura de cores
        gl.glEnable(gl.GL_BLEND)
        
        #configura função de mistura de cores
        gl.glBlendFunc(gl.GL_SRC_ALPHA, gl.GL_ONE_MINUS_SRC_ALPHA)
        
    def paintGL(self):
        
        # configura a cor de background
        gl.glClearColor(0.3, 0.3, 0.3, 1)
        
        # limpa o background com a cor especificada e o buffer de profundidade
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)
        
        # calcula o tempo de execução do programa
        self.currentTime = time.time()
        time_difference = self.currentTime - self.startTime
        
        #atualiza as coordenadas de terreno das esferas animadas
        if(time_difference > 3):
            self.startTime = self.currentTime
            time_difference = 0
            
            self.aux_index = self.terrain_index
            self.index_offset = np.random.randint(-100, high=100, size=(3,2))
            
        self.terrain_index = self.aux_index + (time_difference / 3.0) * self.index_offset
        self.terrain_index = np.clip(self.terrain_index, 0, 300)
        
        # ativa o shader program que usa Phong shading
        self.multiPhongShaderProgram.bind()
        
        #atualiza a posição das luzes
        self.updateLightPosition()
        
        # renderiza o terreno
        self.renderTerrain()
        
        # renderiza as esferas estáticas
        self.renderStaticSpheres()
        
        # desativa o shader program
        self.multiPhongShaderProgram.release()
        
        # ativa o shader program simples
        self.simpleShaderProgram.bind()
        
        # renderizas as esferas representando as luzes
        self.renderMovingSpheres()
        
        # desativa o shader program
        self.simpleShaderProgram.bind()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()
        
    def updateLightPosition(self):
        
        light_position = self.terrainMesh.getPositionAt(int(self.terrain_index[0, 0]), int(self.terrain_index[0, 1]))
        light_position = glm.vec4(light_position[0], light_position[1], light_position[2] + 0.1, light_position[3])
        self.multiPhongShaderProgram.setUniformLightPosition(0, self.viewMatrix * self.globalTransformation * light_position)
        
        light_position = self.terrainMesh.getPositionAt(int(self.terrain_index[1, 0]), int(self.terrain_index[1, 1]))
        light_position = glm.vec4(light_position[0], light_position[1], light_position[2] + 0.1, light_position[3])
        self.multiPhongShaderProgram.setUniformLightPosition(1, self.viewMatrix * self.globalTransformation * light_position)
        
        light_position = self.terrainMesh.getPositionAt(int(self.terrain_index[2, 0]), int(self.terrain_index[2, 1]))
        light_position = glm.vec4(light_position[0], light_position[1], light_position[2] + 0.1, light_position[3])
        self.multiPhongShaderProgram.setUniformLightPosition(2, self.viewMatrix * self.globalTransformation * light_position)
        
    def renderTerrain(self):
        
        #calcula a matriz model-view do terreno
        self.multiPhongShaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * self.globalTransformation)
        self.multiPhongShaderProgram.setUniformModelViewMatrix(self.viewMatrix * self.globalTransformation)
        
        #configura as propriedades do material do terreno
        self.multiPhongShaderProgram.setUniformMaterialColor(np.array([0.5, 0.5, 0.5, 1.0]))
        self.multiPhongShaderProgram.setUniformMaterialAmbient(np.array([0.8, 0.8, 0.8]))
        self.multiPhongShaderProgram.setUniformMaterialDiffuse(np.array([0.8, 0.8, 0.8]))
        self.multiPhongShaderProgram.setUniformMaterialSpecular(np.array([0.1, 0.1, 0.1]))
        self.terrainRenderer.render()
    
    #renderiza as esferas que estão se movendo
    def renderMovingSpheres(self):
        
        #configura a cor da esfera azul
        self.simpleShaderProgram.setUniformColor(np.array([0.0, 0.0, 1.0, 1.0]))
        
        #calcula a matriz model-view da esfera cinza
        sphere_position = self.terrainMesh.getPositionAt(int(self.terrain_index[0, 0]), int(self.terrain_index[0, 1]))
        model_matrix = glm.translate(glm.mat4(), glm.vec3(sphere_position[0], sphere_position[1], sphere_position[2] + 0.1))
        mv_matrix = self.viewMatrix * self.globalTransformation * model_matrix
        self.renderSimpleSphere(mv_matrix)
        
        #configura a cor da esfera verde
        self.simpleShaderProgram.setUniformColor(np.array([0.0, 1.0, 0.0, 1.0]))
        
        #calcula a matriz model-view da esfera vermelha
        sphere_position = self.terrainMesh.getPositionAt(int(self.terrain_index[1, 0]), int(self.terrain_index[1, 1]))
        model_matrix = glm.translate(glm.mat4(), glm.vec3(sphere_position[0], sphere_position[1], sphere_position[2] + 0.1))
        mv_matrix = self.viewMatrix * self.globalTransformation * model_matrix
        self.renderSimpleSphere(mv_matrix)
        
        #configura a cor da esfera vermelha
        self.simpleShaderProgram.setUniformColor(np.array([1.0, 0.0, 0.0, 1.0]))
        
        #calcula a matriz model-view da esfera laranja
        sphere_position = self.terrainMesh.getPositionAt(int(self.terrain_index[2, 0]), int(self.terrain_index[2, 1]))
        model_matrix = glm.translate(glm.mat4(), glm.vec3(sphere_position[0], sphere_position[1], sphere_position[2] + 0.07))
        mv_matrix = self.viewMatrix * self.globalTransformation * model_matrix
        self.renderSimpleSphere(mv_matrix)
    
    #rendenriza as esferas estáticas
    def renderStaticSpheres(self):
        
        scale = glm.scale(glm.mat4(), glm.vec3(1.5, 1.5, 1.5))
        
        #configura as propriedades do material da esfera do fundo
        self.multiPhongShaderProgram.setUniformMaterialColor(np.array([1.0, 1.0, 1.0, 1.0]))
        self.multiPhongShaderProgram.setUniformMaterialAmbient(np.array([0.329412, 0.223529, 0.027451]))
        self.multiPhongShaderProgram.setUniformMaterialDiffuse(np.array([0.780392, 0.568627, 0.113725]))
        self.multiPhongShaderProgram.setUniformMaterialSpecular(np.array([0.992157, 0.941176, 0.807843]))
        self.multiPhongShaderProgram.setUniformMaterialShininess(27.8974)
        
        #calcula a matriz model-view da esfera do fundo
        transl = glm.translate(glm.mat4(), glm.vec3(0.0, 0.3, 1.0))
        mv_matrix = self.viewMatrix * self.globalTransformation * transl * scale
        self.renderMultiPhongSphere(mv_matrix)
        
        #configura as propriedades do material da esfera da direita
        self.multiPhongShaderProgram.setUniformMaterialColor(np.array([1.0, 1.0, 1.0, 0.7]))
        self.multiPhongShaderProgram.setUniformMaterialAmbient(np.array([0.19125, 0.0735, 0.0225]))
        self.multiPhongShaderProgram.setUniformMaterialDiffuse(np.array([0.7038, 0.27048, 0.0828]))
        self.multiPhongShaderProgram.setUniformMaterialSpecular(np.array([0.256777, 0.137622, 0.086014]))
        self.multiPhongShaderProgram.setUniformMaterialShininess(12.8)
        
        #calcula a matriz model-view da esfera da direita
        transl = glm.translate(glm.mat4(), glm.vec3(0.3, 0.0, 1.0))
        mv_matrix = self.viewMatrix * self.globalTransformation * transl * scale
        self.renderMultiPhongSphere(mv_matrix)
        
        #configura as propriedades do material da esfera da esquerda
        self.multiPhongShaderProgram.setUniformMaterialColor(np.array([1.0, 1.0, 1.0, 0.95]))
        self.multiPhongShaderProgram.setUniformMaterialAmbient(np.array([0.135, 0.2225, 0.1575]))
        self.multiPhongShaderProgram.setUniformMaterialDiffuse(np.array([0.54, 0.89, 0.63]))
        self.multiPhongShaderProgram.setUniformMaterialSpecular(np.array([0.316228, 0.316228, 0.316228]))
        self.multiPhongShaderProgram.setUniformMaterialShininess(12.8)
        
        #calcula a matriz model-view da esfera da esquerda
        transl = glm.translate(glm.mat4(), glm.vec3(-0.3, 0.0, 1.0))
        mv_matrix = self.viewMatrix * self.globalTransformation * transl * scale
        self.renderMultiPhongSphere(mv_matrix)
        
        #configura as propriedades do material da esfera da frente
        self.multiPhongShaderProgram.setUniformMaterialColor(np.array([1.0, 1.0, 1.0, 0.8]))
        self.multiPhongShaderProgram.setUniformMaterialAmbient(np.array([0.1, 0.18725, 0.1745]))
        self.multiPhongShaderProgram.setUniformMaterialDiffuse(np.array([0.396, 0.74151, 0.69102]))
        self.multiPhongShaderProgram.setUniformMaterialSpecular(np.array([0.297254, 0.30829, 0.306678]))
        self.multiPhongShaderProgram.setUniformMaterialShininess(12.8)
        
        #calcula a matriz model-view da esfera da frente
        transl = glm.translate(glm.mat4(), glm.vec3(0.0, -0.3, 1.0))
        mv_matrix = self.viewMatrix * self.globalTransformation * transl * scale
        self.renderMultiPhongSphere(mv_matrix)
    
    #renderiza uma esfera
    def renderMultiPhongSphere(self, mv_matrix):
        
        #configura a matriz mvp para renderizar a cena
        self.multiPhongShaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * mv_matrix)
        
        #configura a matriz model-view utilizada no cálculo da iluminação
        self.multiPhongShaderProgram.setUniformModelViewMatrix(mv_matrix)
        
        self.multiPhongSphereRenderer.render()
    
    #renderiza uma esfera
    def renderSimpleSphere(self, mv_matrix):
        
        #configura a matriz mvp para renderizar a cena
        self.simpleShaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * mv_matrix)
        
        self.simpleSphereRenderer.render()
            
    def resizeGL(self, width, height):
        
        gl.glViewport(0, 0, width, height)
        
        # configura a projeção
        aspectRatio = width / height
        self.perspectiveMatrix = glm.perspective(glm.radians(60.0), aspectRatio, 0.1, 50.0)

    def keyPressEvent(self, event):
        super(MyWidget, self).keyPressEvent(event)
        
        step = 0.5
        
        # verifica se foi pressionada a tecla de seta 'para cima'
        if event.key() == QtCore.Qt.Key_Up:
            self.cameraPos.y += step

        # verifica se foi pressionada a tecla de seta 'para baixo'
        elif event.key() == QtCore.Qt.Key_Down:
            self.cameraPos.y -= step
        
        # verifica se foi pressionada a tecla de seta 'para esquerda'
        elif event.key() == QtCore.Qt.Key_Left:
            self.cameraPos.x -= step
        
        # verifica se foi pressionada a tecla de seta 'para direita'
        elif event.key() == QtCore.Qt.Key_Right:
            self.cameraPos.x += step
        
        # verifica se foi pressionada a tecla de seta '-'
        elif event.key() == QtCore.Qt.Key_Minus:
            self.cameraPos.z -= step
        
        # verifica se foi pressionada a tecla de seta '+'
        elif event.key() == QtCore.Qt.Key_Plus:
            self.cameraPos.z += step
        
        # atualiza a câmera
        self.viewMatrix = glm.lookAt(self.cameraPos, self.cameraFront, self.cameraUp)
        
def main():
    import sys

    #Criação de um aplicativo Qt
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    #Especificação do contexto OpenGL
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    #Criação da janela de renderização
    w = MyWidget(glformat)
    w.resize(800, 800)
    w.setWindowTitle('OpenGL example')
    w.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

TerrainMesh --> Trying to open cg/images/heightmap_1.png
opened file: size= (539, 539) format= PNG mode= L


SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [33]:
! jupyter nbconvert --to python 36_Terreno_com_esferas_iluminacao_multiplas_fontes_de_luz.ipynb
%run -i 36_Terreno_com_esferas_iluminacao_multiplas_fontes_de_luz.py

[NbConvertApp] Converting notebook 32_Esferas_iluminacao.ipynb to python
[NbConvertApp] Writing 10068 bytes to 32_Esferas_iluminacao.py
